# **1. Corpus**

## Crawl URLs to extract all internal links 

**XENU Link Sleuth**  
https://home.snafu.de/tilman/xenulink.html 

*Le logiciel XENU Link Sleuth a finalement été retenu pour cette tâche* 

## Scrape textual data from crawled URLs
**BeautifulSoup HTML Parser**  
Réf : https://realpython.com/python-web-scraping-practical-introduction/

In [9]:
path = 'D:/1-XENU_crawl/csv_html_pdfs/'
acteur = "cisss_lanaudiere"

La liste des URLs à scrapper pour chaque corpus est contenue dans un fichier CSV. 
On commence donc par lire le CSV pour extraire nos URLS.

In [10]:
from pandas import *

regex = '.*png.*|.*jpeg.*|.*jpg.*|.*docx.*|.*js.*|.*font.*|.*gif.*|.*formulaire.*|.*?f%5B0%5D.*|.*img.*|.*%5Bfilter%.*|.*css.*|.*scripts.*|.*zip.*|.*xlsx.*|.*cms.*|.*pdf.*|.*/images/.*'


# encoding= 'ISO-8859-1' "utf-8"
path = 'D:/1-XENU_crawl/csv_html_pdfs/'
with open(path + acteur + '.csv', encoding='UTF-8') as f:
    csv = read_csv(f, sep=';')
    # Nettoyer ce qui ne devrait pas se trouver là
    csv = csv[~csv["Address"].str.contains(regex)] 

In [11]:
csv

,Address,Type,Title,Charset,Description
0,https://www.cisss-lanaudiere.gouv.qc.ca/,text/html,CISSS de Lanaudière: CISSS de Lanaudière,utf-8,Services de santé et services sociaux - CISSS ...
1,https://www.cisss-lanaudiere.gouv.qc.ca/employes/,text/html,Employés: CISSS de Lanaudière,utf-8,Services de santé et services sociaux - CISSS ...
2,https://www.cisss-lanaudiere.gouv.qc.ca/accuei...,text/html,Coordonnées: CISSS de Lanaudière,utf-8,Services de santé et services sociaux - CISSS ...
3,https://www.cisss-lanaudiere.gouv.qc.ca/profes...,text/html,Ressources pour les professionnels: CISSS de L...,utf-8,Services de santé et services sociaux - CISSS ...
4,https://www.cisss-lanaudiere.gouv.qc.ca/englis...,text/html,Services to the English-speaking population: C...,utf-8,Services de santé et services sociaux - CISSS ...
...,...,...,...,...,...
5082,https://www.cisss-lanaudiere.gouv.qc.ca/docume...,text/html,&lsaquo; Précédent,utf-8,NaN
5083,https://www.cisss-lanaudiere.gouv.qc.ca/docume...,text/html,12,utf-8,NaN
5084,https://www.cisss-lanaudiere.gouv.qc.ca/docume...,text/html,13,utf-8,NaN
5087,https://www.cisss-lanaudiere.gouv.qc.ca/docume...,text/html,7,utf-8,NaN


In [12]:
#liste = csv[csv['Type'] != 'application/pdf'] # On va scraper les PDFs avec une autre librairie que BeautifulSoup

liste = csv['Address'].tolist()
fr = csv[~csv["Address"].str.contains('/en/')]['Address'].tolist() # Données en français
en = csv[csv["Address"].str.contains('/en/')]['Address'].tolist() # Données en anglais
print("On va tenter d'aspirer {} pages Web".format(len(liste)))

On va tenter d'aspirer 1191 pages Web


In [13]:
import requests, re, ssl, os, sys, pandas as pd
from bs4 import BeautifulSoup
#from requests.packages.urllib3.util.retry import Retry

def getTextURL(url):    
    html = requests.get(url, headers = {'User-Agent': 'My User Agent 1.0'}, verify=False)
    html.encoding = 'utf-8'
    html = html.text

    soup = BeautifulSoup(html, "html.parser")
    tags_to_remove = ['head', 'header', 'script', 'footer', 'nav', 'form'] # Enlever 'form' pour le site du CHU Ste Justine

    # Classes CSS spécifiques aux différents sites
    attr_to_remove = ['div[class="contenu-fluide piv"]', 'div[role="navigation"]', 'div[class="section__wrapper section__wrapper--padding tac grid--inline-block"]', 'div[id="slidebox"]',
    'div[class="col-md-12 mise-a-jour"]', 'h1[class="sr-only"]', 'a[class="cd-top js-cd-top"]', 'div[class="nocontent"]', 'p[class="footer-lien-resonances"]', 'p[class="suivre"]',
    'section[class="field field-name-field-date-de-mise-jour field-type-datetime field-label-inline clearfix view-mode-full"]', 'a[class="active"]', 'p[class="footer-resonances"]',
    'div[class="item-list item-list-pager"]', 'ul[class="pub-solr-sub-menu"]', 'a[href="#main-content"]', 'div[id="block-sociauxcrchum"]', 'div[class="visually-hidden"]',
    'div[id="Breadcrumb"]', 'div[id="pageInfo"]', 'div[id="breadcrumb"]', 'div[class="pagesCreation"]', 'a[href="#contenu"]', 'div[class="bandeau"]',
    'div[id="seeAlso"]', 'a[href="/nous-ecrire.aspx"]', 'li[class="CMSListMenuLI"]', 'li[class="CMSListMenuLI navFirst"]', 'li[class="CMSListMenuLI navLast"]',
    'div[class="alert alert-danger"]', 'span[class="alertoverflow"]', 'div[class="alert alert-warning alert-dismissible"]', 'ul[class="menu"]', 'div[id="letters-filter"]',
    'ul[class="pager"]', 'a[href="#main-menu"]', 'ul[class="custom_menu"]', 'h2[class="element-invisible"]', 'div[class="breadcrumb"]',
    'a[class="all-cta"]', 'div[class="sub-menu-inner container"]', 'div[class="fixed-dk-nav"]', 'div[class="fixed-dk-nav-container"]', 'div[class="container-inner"]',
    'div[class="socials"]', 'div[class="breadcrumbs"]', 'a[class="btn-print"]', 'ul[class="list-buttons"]', 'p[class="visually-hidden"]', 
    'a[class="back-to-top"]', 'a[class="sr-only sr-only-focusable"]', 'ol[class="breadcrumb"]', 'div[class="container-fluid piv_bas"]', 'div[class="col-12 formBasPage"]',
    'div[class="container-fluid rangee-footer"]', 'a[class="visuallyHidden passerContenu"]', 'div[id="bandeau-alerte"]', 'div[class="menu-sec-wrapper col-12 col-lg-12"]',
    'a[href="#layout-content"]', 'div[class="paragraph feedback"]', 'p[class="last-update"]', 'ul[class="footer__menu--list"]', 'div[class="footer__info"]',
    'section[class="hello-bar"]', 'section[class="breadcrumb"]', 'div[class="menu-page"]', 'div[class="no-print menu-non-voyant"]', 'div[class="navigation"]',
    'div[class="pure-bloc pure-u-1 pure-u-md-1-3 pure-u-lg-1-4 side-menu"]', 'div[class="pied"]', 'div[class="social"]', 'div[class="piv-bas"]',
    'div[class="partage"]', 'div[class="pied-print no-screen"]', 'div[class="carte dynamic-carte-interactive-display ui-carte-panel"]',
    'div[id="piv"]', 'ul[class="social-nav top-bar-social"]', 'div[class="sidebar"]', 'ul[class="side-menu"]', 'div[class="zoom-button-wrapper"]',
    'a[href="#maincontent"]', 'a[href="#content"]', 'p[id="breadcrumbs"]', 'div[class="mega-menu-wrap"]', 'div[class="menu_2"]', 'div[class="welcome"]',
    'div[class="header_two"]', 'div[class="footer"]', 'div[class="footer-wrapper"]', 'div[class="custom-accessibility-tools js-only"]', 'section[role="navigation"]']
    
    for t in tags_to_remove:
        tags = soup.find_all(t)
        for tag in tags:
            tag.decompose()

    for t in attr_to_remove:
        attr = soup.select(t)
        for a in attr:
            a.decompose()


    data = soup.get_text(separator=' ').replace("\n", " ").replace("\r", " ") 
    data = re.compile(r"\s+").sub(" ", data).strip()
    
    return data



In [14]:
def scrape_list(x):
    output = []
    for site in x: 
        try: 
            text = getTextURL(site)
            if not '���' in text:
                output.append({'url': site, 'text':text})
        except Exception as e:
            print("ERROR " + " - " + site)
            print(e)

    return output

In [15]:
sites_fr = scrape_list(fr)
if(len(en) > 0):
    sites_en = scrape_list(en)

c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.cisss-lanaudiere.gouv.qc.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.cisss-lanaudiere.gouv.qc.ca'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.cisss-lanaudiere.gouv.qc.ca'. Adding certificate verification is strongly advised. See: ht

In [16]:
output_path = 'D:/1-XENU_crawl/scraping_07-2022/'

pd.DataFrame(sites_fr).to_csv(output_path + acteur + '.csv', escapechar='/')
if(len(en) > 0):
    pd.DataFrame(sites_en).to_csv(output_path + acteur + '_en.csv', escapechar='/')